https://it.mathworks.com/help/matlab/matlab_external/call-user-script-and-function-from-python.html

https://it.mathworks.com/help/matlab/matlab_external/call-matlab-functions-from-python.html

https://it.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html

In [7]:
import sys
sys.path.append("/usr/local/lib/python2.7/dist-packages/matlab")
#sys.path.append("/opt/anaconda/miniconda3/lib/python3.6/site-packages/matlab")
import matlab
from matlab import engine

In [9]:
eng=engine.start_matlab()
#V=eng.tensor_matlab(path)
#list_cc=eng.CC_matlab(V,R_f,nb_s)

In [11]:
def CC_matlab(df_classified,vectorizer_new=None,n_features=1000,R_f=50, nb_s=5, path='CC_matlab/'):
    """returns the list of n_topics, their mean core consistencies and their variances."""
    import sys
    sys.path.append("/usr/local/lib/python2.7/dist-packages/matlab")
    #sys.path.append("/opt/anaconda/miniconda3/lib/python3.6/site-packages/matlab")
    import matlab
    from matlab import engine
    eng=matlab.engine.start_matlab()
    # Feed the vectorizer with all the words in the dataset. Counts is the tweet/term matrix.
    # fit_transform: fit first (build the features list with the relevant words)
    # then transform: build the tweet/term matrix with the relevant tokens.
    if not vectorizer_new:
        print 'No vectorizer defined. Returning None'
        return None
    name_matlab='CC_matlab/Matr{}_'.format(n_features)
    snapshots=df_classified.columns.tolist()[0:-1]        
    #For every snapshot taken
    ct=0
    for month in snapshots[start:stop]:
        ct+=1
        This_Month=df_classified[month].tolist()
        # Learn the vocabulary dictionary and return term-document matrix.
        counts = vectorizer_new.transform(This_Month)
        #Transform a count matrix to a normalized tf-idf representation. 
        #(i.e terms with frequencies too hi or lo are removed)
        # Weights are indexed by (postID, term): weight
        tfidf = TfidfTransformer().fit_transform(counts)
        savemat(name_matlab+str(ct), {'tfidf':tfidf})
    V=eng.tensor_matlab(name_matlab)
    list_idn,list_meancc,list_varcc=eng.CC_matlab(V,R_f,nb_s,nargout=3)
    #A,B,C 
    return V,list_idn,list_meancc,list_varcc

def NTF_matlab(V,R):
    """returns the NTF factorization with R topics from a TF-IDF vector V built by matlab."""
    import sys
    sys.path.append("/usr/local/lib/python2.7/dist-packages/matlab")
    #sys.path.append("/opt/anaconda/miniconda3/lib/python3.6/site-packages/matlab")
    import matlab
    from matlab import engine
    eng=matlab.engine.start_matlab()
    Xapprox_ks,A,B,C=eng.NTF_matlab(V,R,nargout=4)
    return Xapprox_ks,A,B,C